In [300]:
import pandas
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression as lr
from sklearn.metrics import classification_report
from sklearn import metrics

In [301]:
data = pandas.read_csv("CW - part 2/data/processed.pkl.gz")

In [302]:
# clone database and convert CDR column to bool
data1 = data.copy()
data1["CDR"] = data1['CDR'].map({'very mild': True, 'mild': True, 'moderate': True, 'none': False})

In [303]:
data.insert(13,"has_dementia?", data1["CDR"])

In [304]:
data

,ID,MRI_visit,delay,sex,hand,age,YOE,SES,MMSE,CDR,eTIV,nWBV,ASF,has_dementia?
0,1,MR1,0,M,R,87,14,2.0,27.0,none,1987.0,0.696,0.883,False
1,1,MR2,457,M,R,88,14,2.0,30.0,none,2004.0,0.681,0.876,False
2,2,MR1,0,M,R,75,12,0.0,23.0,very mild,1678.0,0.736,1.046,True
3,2,MR2,560,M,R,76,12,0.0,28.0,very mild,1738.0,0.713,1.010,True
4,2,MR3,1895,M,R,77,12,0.0,22.0,very mild,1698.0,0.701,1.034,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
368,149,MR2,842,M,R,81,16,1.0,28.0,very mild,1693.0,0.694,1.037,True
369,149,MR3,2297,M,R,82,16,1.0,26.0,very mild,1688.0,0.675,1.040,True
370,150,MR1,0,F,R,61,13,2.0,30.0,none,1319.0,0.801,1.331,False
371,150,MR2,763,F,R,62,13,2.0,30.0,none,1327.0,0.796,1.323,False


In [305]:
y = data["has_dementia?"]
x = data.drop("has_dementia?", axis="columns")

In [306]:
le_CDR = LabelEncoder()
le_sex = LabelEncoder()
le_hand = LabelEncoder()
le_MRI_visit = LabelEncoder()

In [307]:
x['CDR_num'] = le_CDR.fit_transform(x['CDR'])
x['sex_num'] = le_sex.fit_transform(x['sex'])
x['hand_num'] = le_hand.fit_transform(x['hand'])
x['MRI_visit_num'] = le_MRI_visit.fit_transform(x['MRI_visit'])

In [308]:
x = x.drop(['CDR','sex','hand','MRI_visit', 'ID', 'CDR_num'],axis='columns')

In [413]:
dt = tree.DecisionTreeClassifier(max_depth=3)

In [414]:
predicted = dt.fit(x,y).predict(x)

In [415]:
report = metrics.classification_report(y, predicted)

In [416]:
print(report)

              precision    recall  f1-score   support

       False       0.79      0.96      0.87       206
        True       0.93      0.69      0.79       167

    accuracy                           0.84       373
   macro avg       0.86      0.82      0.83       373
weighted avg       0.85      0.84      0.83       373



In [417]:
cross_val_score(dt, x, y, scoring="accuracy", cv=5).mean()

0.780072072072072

In [418]:
text = tree.export_text(dt, feature_names=x.columns.tolist())

In [419]:
print(text)

|--- MMSE <= 27.50
|   |--- MMSE <= 26.50
|   |   |--- nWBV <= 0.76
|   |   |   |--- class: True
|   |   |--- nWBV >  0.76
|   |   |   |--- class: True
|   |--- MMSE >  26.50
|   |   |--- eTIV <= 1299.00
|   |   |   |--- class: False
|   |   |--- eTIV >  1299.00
|   |   |   |--- class: True
|--- MMSE >  27.50
|   |--- sex_num <= 0.50
|   |   |--- MMSE <= 28.50
|   |   |   |--- class: False
|   |   |--- MMSE >  28.50
|   |   |   |--- class: False
|   |--- sex_num >  0.50
|   |   |--- ASF <= 1.29
|   |   |   |--- class: False
|   |   |--- ASF >  1.29
|   |   |   |--- class: True

